In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import edunn as nn
from edunn import utils

# Descenso de gradiente

El descenso de gradiente es una técnica de optimización simple pero efectiva para entrenar modelos derivables.

En cada iteración del algoritmo, se calcula la derivada del error respecto a cada uno de los parámetros `δEδp`, actualizan los pesos en la dirección contraria al gradiente. Esta actualización está mediada por el parámetro `α` que indica la tasa de aprendizaje. 

El algoritmo de descenso de gradiente es simple:

```python
for i in range(iteraciones):
    for p in model.parameters()
        # usamos p[:] para modificar los valores de p
        # y no crear una nueva variable
        p[:] = p - α * δEδp(x,y)
```

Este pseudocódigo obvia algunas partes engorrosas. En particular, la iteración sobre los valores de entrada `x` y salida `y` de los ejemplos, en su versión por `batches`, y el cálculo del error y las derivadas `δEδp`. 

La librería `edunn` cuenta con la clase `BatchedGradientOptimizer` que se encarga de eso, y nos permite implementar un optimizador de forma muy simple creando una subclase de ella, e implementando el método `optimize_batch`, en donde solo tenemos preocuparnos por optimizar el modelo utilizando las derivadas calculadas con un batch del conjunto de datos. 

Para este ejercicio, hemos creado la clase `GradientDescent`, que subclasifica a `BatchedGradientOptimizer`. Implemente, entonces, la parte crucial del método `optimize_batch` de `GradientDescent`, para que actualice los parámetros en base a los los gradientes ya calculados.

Para probar este optimizador, vamos a utilizar un modelo falso y error falso que nos permitan controlar de manera la entrada al optimizador. La flexiblidad de la clase `Model` de `edunn` permite hacer esto muy fácilmente creando las clases `FakeModel` y `FakeError`, que ignoran realmente sus entradas y salidas, y solo sirven para que `FakeModel` inicialice 2 parámetros con valore 0 y retorne `[-1,1]` como derivada para ellos.

In [ ]:
#Modelo falso con un vector de parámetros con valor inicial [0,0] y gradientes que siempre son [1,-11]
model = nn.FakeModel(parameter=np.array([0,0]),gradient=np.array([1, -1]))
# función de error falso cuyo error es siempre 1 y las derivadas también
error = nn.FakeError(error=1,derivative_value=1)

# Conjunto de datos falso, que no se utilizará realmente
fake_samples = 3
fake_x = np.random.rand(fake_samples,10)
fake_y = np.random.rand(fake_samples,5)

# Optimizar el modelo por 1 época con lr=2
optimizer = nn.GradientDescent(batch_size=fake_samples,epochs=1,lr=2,shuffle=False)
history = optimizer.optimize(model,fake_x,fake_y,error,verbose=False)
expected_parameters=np.array([-2,2])
utils.check_same(expected_parameters,model.get_parameters()["parameter"])

# Optimizar el modelo por 1 época *adicional* con lr=2
history = optimizer.optimize(model,fake_x,fake_y,error,verbose=False)
expected_parameters=np.array([-4,4])
utils.check_same(expected_parameters,model.get_parameters()["parameter"])
    
# Optimizar el modelo por 3 épocas más, ahora con con lr=1    
optimizer = nn.GradientDescent(batch_size=fake_samples,epochs=3,lr=1,shuffle=False)
history = optimizer.optimize(model,fake_x,fake_y,error,verbose=False)
expected_parameters=np.array([-7,7])
utils.check_same(expected_parameters,model.get_parameters()["parameter"])    
    

# Entrenamiento de un modelo de Regresión Lineal con Descenso de gradiente

Ahora que tenemos todos los elementos, podemos definir y entrenar nuestro primer modelo `RegresionLineal` para estimar el precio de casas utilizando el conjunto de datos de [Casas de Boston](https://www.kaggle.com/c/boston-housing)

In [ ]:
import edunn as nn
import numpy as np
from edunn import metrics,datasets

x,y=datasets.load_regression("boston")
x = (x-x.mean(axis=0))/x.std(axis=0)
n, din = x.shape
n, dout = y.shape
print("Dataset sizes:", x.shape,y.shape)

#Red con dos capas lineales
model = nn.LinearRegression(din,dout)
error = nn.MeanError(nn.SquaredError())
optimizer = nn.GradientDescent(lr=0.001,epochs=1000,batch_size=32)

# Algoritmo de optimización
history = optimizer.optimize(model,x,y,error)
nn.plot.plot_history(history,error_name=error.name)


print("Error del modelo:")
y_pred=model.forward(x)
metrics.regression_summary(y,y_pred)
nn.plot.regression1d_predictions(y,y_pred)

# Comparación con sklearn

Como verificación adicional, calcularemos los parámetros óptimos de un modelo de regresión lineal con sklearn, y visualizamos los resultados. El error debería ser similar al de nuestro modelo (RMSE=3.27 o 3.28).

In [ ]:
from sklearn import linear_model
model=linear_model.LinearRegression()
model.fit(x,y)
y_pred=model.predict(x)
print("Error del modelo:")
metrics.regression_summary(y,y_pred)
print()

nn.plot.regression1d_predictions(y,y_pred)
